In [40]:
import os

In [41]:
%pwd

'd:\\Projects'

In [51]:
os.chdir("../Projects/kidney-disease-classification")

In [52]:
%pwd

'd:\\Projects\\kidney-disease-classification'

In [53]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [54]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [55]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir 
        )

        return data_ingestion_config

In [56]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [57]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [58]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-23 02:41:50,694: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-23 02:41:50,721: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-23 02:41:50,735: INFO: common: created directory at: artifacts]
[2025-04-23 02:41:50,745: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-23 02:41:50,749: INFO: 3226479738: Downloading data from https://drive.google.com/file/d/1t0fXtg3ibJ59KWYeUT_paKpguzJqLp-v/view?usp=sharing into file artifacts/data_ingestion/kidneydataset.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1t0fXtg3ibJ59KWYeUT_paKpguzJqLp-v
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1t0fXtg3ibJ59KWYeUT_paKpguzJqLp-v&confirm=t&uuid=f6faf4f2-2a60-4512-8e31-a91e4f7e8364
To: d:\Projects\kidney-disease-classification\artifacts\data_ingestion\kidneydataset.zip
100%|██████████| 940M/940M [01:54<00:00, 8.19MB/s] 


[2025-04-23 02:43:53,128: INFO: 3226479738: Downloaded data from https://drive.google.com/file/d/1t0fXtg3ibJ59KWYeUT_paKpguzJqLp-v/view?usp=sharing into file artifacts/data_ingestion/kidneydataset.zip]
